# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("nhl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

########################################################################

# cleaning the cities dataframe
cities["NHL"]=cities["NHL"].apply(lambda x: re.sub("\[.+\]", "", x))
cities=cities[(cities['NHL'] != '—') & (cities['NHL'] != '')] #drop cities without NHL team
cities=cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']] # get important columns
cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace = True)

multiples_teams={"Rangers Islanders Devils": "Rangers,Islanders,Devils", "Kings Ducks": "Kings,Ducks"} #create a dictionarie to separe the rows with multiple teams
cities["NHL"] = cities["NHL"].replace(multiples_teams)
cities["NHL"] = cities["NHL"].apply(lambda x: x.split(",")) #split the text in multiple items accroding to the comma spearator
cities = cities.explode("NHL") # explode the multiples itemes create in the previus step in multiple rows

########################################################################

# cleaning nhl_df dataframe
nhl_df=nhl_df[nhl_df['year']==2018] #get 2018 data
nhl_df=nhl_df[['team', 'W', 'L']] # get win, loses columns
nhl_df['team'] = nhl_df['team'].map(lambda x: re.sub("\*", "", x)) # clear index labels having '*'

########################################################################

#make a dictionary with the names of the teams (the words that are in list B NHL column):
#https://stackoverflow.com/questions/73049057/comparing-a-list-to-another-nested-list-in-python
#https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists

A = nhl_df['team'].map(lambda x: re.split(r" ", x))
B = cities['NHL'].apply(lambda x: re.split(r" ", x))

def flatten(x):
    for e in x:
        if isinstance(e, list):
            yield from flatten(e)
        else:
            yield e 
            
newlist=[x for x in flatten(A) if x in flatten(B)]

#nhl_df['team'] = nhl_df.apply(lambda x: ' '.join([i for i in newlist if i.lower() in x['team'].lower().split()]), axis=1) #apply dictionarie to column teams
nhl_df['team'] = nhl_df['team'].apply(lambda x: ' '.join(i for i in x.split() if i in newlist)) #The difference with the previus aproach is that this one keep the order of the words.
nhl_df=nhl_df[(nhl_df['team'] != '')]

########################################################################

# merge the dataframes

df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
df = df.astype({'W': int, 'L': int, 'Population': int}) # redifine data type for calculations
df['W/L rate'] = df['W']/(df['W']+df['L'])
df=df.groupby('Metropolitan area').agg({'W/L rate': np.mean, 'Population': np.mean, 'W':np.mean, 'L':np.mean})

########################################################################

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L rate'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012486162921209909

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("nba.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

########################################################################

# cleaning the cities dataframe
cities["NBA"]=cities["NBA"].apply(lambda x: re.sub("\[.+\]", "", x))
cities=cities[(cities['NBA'] != '—') & (cities['NBA'] != '')] #drop cities without NHL team
cities=cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']] # get important columns
cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace = True)

multiples_teams={"Knicks Nets": "Knicks,Nets", "Lakers Clippers": "Lakers,Clippers"} #create a dictionarie to separe the rows with multiple teams
cities["NBA"] = cities["NBA"].replace(multiples_teams)
cities["NBA"] = cities["NBA"].apply(lambda x: x.split(",")) #split the text in multiple items accroding to the comma spearator
cities = cities.explode("NBA") # explode the multiples itemes create in the previus step in multiple rows

########################################################################

# cleaning nba_df dataframe
nba_df=nba_df[nba_df['year']==2018] #get 2018 data
nba_df=nba_df[['team', 'W', 'L']] # get win, loses columns
nba_df['team'] = nba_df['team'].map(lambda x: re.sub("(\*)*\s\(\d+\)", "", x)) # clear index labels having '*' or (0-9)

########################################################################

#make a dictionary with the names of the teams (the words that are in list B NHL column):
#https://stackoverflow.com/questions/73049057/comparing-a-list-to-another-nested-list-in-python
#https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists

A = nba_df['team'].map(lambda x: re.split(r" ", x))
B = cities['NBA'].apply(lambda x: re.split(r" ", x))

def flatten(x):
    for e in x:
        if isinstance(e, list):
            yield from flatten(e)
        else:
            yield e 
            
newlist=[x for x in flatten(A) if x in flatten(B)]

nba_df['team'] = nba_df['team'].apply(lambda x: ' '.join(i for i in x.split() if i in newlist)) #apply dictionarie to column teams
nba_df=nba_df[(nba_df['team'] != '')]

########################################################################

# merge the dataframes

df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
df = df.astype({'W': int, 'L': int, 'Population': int}) # redifine data type for calculations
df['W/L rate'] = df['W']/(df['W']+df['L'])
df=df.groupby('Metropolitan area').agg({'W/L rate': np.mean, 'Population': np.mean, 'W':np.mean, 'L':np.mean})

########################################################################

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L rate'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("mlb.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

########################################################################

# cleaning the cities dataframe
cities["MLB"]=cities["MLB"].apply(lambda x: re.sub("\[.+\]", "", x))
cities=cities[(cities['MLB'] != '—') & (cities['MLB'] != '')] #drop cities without NHL team
cities=cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']] # get important columns
cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace = True)

multiples_teams={"Yankees Mets": "Yankees,Mets", "Dodgers Angels": "Dodgers,Angels", "Giants Athletics":"Giants,Athletics", "Cubs White Sox":"Cubs,White Sox"} #create a dictionarie to separe the rows with multiple teams
cities["MLB"] = cities["MLB"].replace(multiples_teams)
cities["MLB"] = cities["MLB"].apply(lambda x: x.split(",")) #split the text in multiple items accroding to the comma spearator
cities = cities.explode("MLB") # explode the multiples itemes create in the previus step in multiple rows

########################################################################

# cleaning mlb_df dataframe
mlb_df=mlb_df[mlb_df['year']==2018] #get 2018 data
mlb_df=mlb_df[['team', 'W', 'L']] # get win, loses columns

########################################################################

#make a dictionary with the names of the teams (the words that are in list B NHL column):
#https://stackoverflow.com/questions/73049057/comparing-a-list-to-another-nested-list-in-python
#https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists

A = mlb_df['team'].map(lambda x: re.split(r" ", x))
B = cities['MLB'].apply(lambda x: re.split(r" ", x))

def flatten(x):
    for e in x:
        if isinstance(e, list):
            yield from flatten(e)
        else:
            yield e 
            
newlist=[x for x in flatten(A) if x in flatten(B)]

mlb_df['team'] = mlb_df['team'].apply(lambda x: ' '.join(i for i in x.split() if i in newlist)) #apply dictionarie to column teams.
mlb_df=mlb_df[(mlb_df['team'] != '')]

########################################################################

# merge the dataframes

df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
df = df.astype({'W': int, 'L': int, 'Population': int}) # redifine data type for calculations
df['W/L rate'] = df['W']/(df['W']+df['L'])
df=df.groupby('Metropolitan area').agg({'W/L rate': np.mean, 'Population': np.mean, 'W':np.mean, 'L':np.mean})

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L rate'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("nfl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

########################################################################

# cleaning the cities dataframe
cities["NFL"]=cities["NFL"].apply(lambda x: re.sub("\[.+\]", "", x))
cities= cities[cities['NFL'].str.contains("[a-zA-Z]").fillna(False)]  #drop cities without any letter entry in NHL
cities=cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']] # get important columns
cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace = True)

multiples_teams={"Giants Jets": "Giants,Jets", "Rams Chargers": "Rams,Chargers", "49ers Raiders":"49ers,Raiders"} #create a dictionarie to separe the rows with multiple teams
cities["NFL"] = cities["NFL"].replace(multiples_teams)
cities["NFL"] = cities["NFL"].apply(lambda x: x.split(",")) #split the text in multiple items accroding to the comma spearator
cities = cities.explode("NFL") # explode the multiples itemes create in the previus step in multiple rows

########################################################################

# cleaning nfl_df dataframe
nfl_df=nfl_df[nfl_df['year']==2018] #get 2018 data
nfl_df=nfl_df[['team', 'W', 'L']] # get win, loses columns
nfl_df['team'] = nfl_df['team'].map(lambda x: re.sub("(\*|\+)", "", x)) # clear index labels having '*' or +

########################################################################

#make a dictionary with the names of the teams (the words that are in list B NHL column):
#https://stackoverflow.com/questions/73049057/comparing-a-list-to-another-nested-list-in-python
#https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists

A = nfl_df['team'].map(lambda x: re.split(r" ", x))
B = cities['NFL'].apply(lambda x: re.split(r" ", x))

def flatten(x):
    for e in x:
        if isinstance(e, list):
            yield from flatten(e)
        else:
            yield e 
            
newlist=[x for x in flatten(A) if x in flatten(B)]

nfl_df['team'] = nfl_df['team'].apply(lambda x: ' '.join(i for i in x.split() if i in newlist)) #apply dictionarie to column teams.
nfl_df=nfl_df[(nfl_df['team'] != '')]

########################################################################

# merge the dataframes

df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
df = df.astype({'W': int, 'L': int, 'Population': int}) # redifine data type for calculations
df['W/L rate'] = df['W']/(df['W']+df['L'])
df=df.groupby('Metropolitan area').agg({'W/L rate': np.mean, 'Population': np.mean, 'W':np.mean, 'L':np.mean})

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L rate'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349409

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

########################################################################

#def a function that get the data as we needed

def get_df(BIG4):
    
    cities=pd.read_html("wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]] 
    big4_df=pd.read_csv(BIG4.lower()+'.csv')
    
# cleaning the cities dataframe
    cities[BIG4]=cities[BIG4].apply(lambda x: re.sub("\[.+\]", "", x))
    cities= cities[cities[BIG4].str.contains("[a-zA-Z]").fillna(False)]  #drop cities without any letter entry in NHL
    cities=cities[['Metropolitan area', 'Population (2016 est.)[8]', BIG4]] # get important columns
    cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace = True)

    multiple_teams={"Rangers Islanders Devils": "Rangers,Islanders,Devils", "Kings Ducks": "Kings,Ducks", #NHL\ 
                    "Knicks Nets": "Knicks,Nets", "Lakers Clippers": "Lakers,Clippers", # NBA\ 
                    "Yankees Mets": "Yankees,Mets", "Dodgers Angels": "Dodgers,Angels", "Giants Athletics":"Giants,Athletics", "Cubs White Sox":"Cubs,White Sox", # MLB\ 
                    "Giants Jets": "Giants,Jets", "Rams Chargers": "Rams,Chargers", "49ers Raiders":"49ers,Raiders"} #NFL
    cities[BIG4] = cities[BIG4].replace(multiple_teams)
    cities[BIG4] = cities[BIG4].apply(lambda x: x.split(",")) #split the text in multiple items accroding to the comma spearator
    cities = cities.explode(BIG4) # explode the multiples itemes create in the previus step in multiple rows

########################################################################

# cleaning big4_df dataframe
    big4_df=big4_df[big4_df['year']==2018] #get 2018 data
    big4_df=big4_df[['team', 'W', 'L']] # get win, loses columns
    big4_df['team'] = big4_df['team'].map(lambda x: re.sub("\*|\+|\s\(\d+\)", "", x)) # clear index labels having '*'

########################################################################

#make a dictionary with the names of the teams (the words that are in list B NHL column):
#https://stackoverflow.com/questions/73049057/comparing-a-list-to-another-nested-list-in-python
#https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists

    A = big4_df['team'].map(lambda x: re.split(r" ", x))
    B = cities[BIG4].apply(lambda x: re.split(r" ", x))

    def flatten(x):
        for e in x:
            if isinstance(e, list):
                yield from flatten(e)
            else:
                yield e 
            
    newlist=[x for x in flatten(A) if x in flatten(B)]

    big4_df['team'] = big4_df['team'].apply(lambda x: ' '.join(i for i in x.split() if i in newlist)) #The difference with the previus aproach is that this one keep the order of the words.
    big4_df=big4_df[(big4_df['team'] != '')]

########################################################################

# merge the dataframes

    df = pd.merge(cities, big4_df, left_on=BIG4, right_on="team")
    df = df.astype({'W': int, 'L': int, 'Population': int}) # redifine data type for calculations
    df['W/L rate'] = df['W']/(df['W']+df['L'])
    df=df.groupby('Metropolitan area').agg({'W/L rate': np.mean, 'Population': np.mean, 'W':np.mean, 'L':np.mean})
    
    return df

########################################################################


def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in sports:
        for j in sports:
            if i!=j:
                Mi=get_df(i)
                Mj=get_df(j)
                Mi=Mi['W/L rate']
                Mj=Mj['W/L rate']
                merge=pd.merge(Mi,Mj,how='inner',left_index=True,right_index=True)
                #print(merge)
                p_values.loc[i, j]=stats.ttest_rel(merge['W/L rate_x'],merge['W/L rate_y'])[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN


In [7]:
import dataframe_image as dfi
dfi.export(sports_team_performance(), "pvalues.png", table_conversion='matplotlib')

findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: 